This notebook is an example of creating and using FHI-Aims input. It runs Oximachine to determine metal center oxidation states, and prepares input files for calculation based on possible spin permutations. It is not documented and not maintained, meaning any routines used need to be checked for validity before use. 

In [6]:
from ase.io import read, write
import os
from ase.calculators.aims import Aims, AimsCube
from oximachinerunner import OximachineRunner
import itertools

In [2]:
os.environ['AIMS_SPECIES_DIR'] = '/Users/matthiasgolomb/Codes/FHIaims/species_defaults/defaults_2020/light'

calc = Aims(xc=('hse06',0.25),
#            hybrid_xc_coeff=0.5,
        hse_unit='B',
        sc_accuracy_etot=1e-4,
        sc_accuracy_eev=1e-2,
        sc_accuracy_rho=1e-5,
        sc_accuracy_forces=1e-2,
        relax_geometry=('trm', 1e-2),
        relax_unit_cell='full',
        collect_eigenvectors='.false.',
#            spin='collinear',
#            charge=-1,
#            fixed_spin_moment=1.0,
#            default_initial_moment=1.0,
        relativistic=('atomic_zora','scalar'),
        k_grid=(1,1,1))

for filename in os.listdir('./'):
    if filename.startswith('geo'):
        filename_popped = filename.replace('geometry_','')
        filename_popped = filename_popped.replace('.in','')
        print(os.getcwd())
        struct = read(filename)
        struct.calc = calc
        if not os.path.exists('./QMOF'):
            os.makedirs('./QMOF')
        if not os.path.exists('./QMOF/{filename}'.format(filename=filename_popped)):
            os.makedirs('./QMOF/{filename}'.format(filename=filename_popped))
        if not os.path.exists('./QMOF/{filename}/Pristine'.format(filename=filename_popped)):
            os.makedirs('./QMOF/{filename}/Pristine'.format(filename=filename_popped))
        if not os.path.exists('./QMOF/{filename}/Pristine/Relaxation'.format(filename=filename_popped)):
            os.makedirs('./QMOF/{filename}/Pristine/Relaxation'.format(filename=filename_popped))
        os.chdir('./QMOF/{filename}/Pristine/Relaxation'.format(filename=filename_popped))
        print(os.getcwd())
        calc.write_input(struct)
                 
        os.chdir('../../../..')

/Users/matthiasgolomb/Library/CloudStorage/Box-Box/QMOF_MPContrib/BatteryMOFs_filter/Structures_all
/Users/matthiasgolomb/Library/CloudStorage/Box-Box/QMOF_MPContrib/BatteryMOFs_filter/Structures_all/QMOF/FeH4C14S2N6/Pristine/Relaxation
/Users/matthiasgolomb/Library/CloudStorage/Box-Box/QMOF_MPContrib/BatteryMOFs_filter/Structures_all
/Users/matthiasgolomb/Library/CloudStorage/Box-Box/QMOF_MPContrib/BatteryMOFs_filter/Structures_all/QMOF/Sc4H28C36N8O24/Pristine/Relaxation
/Users/matthiasgolomb/Library/CloudStorage/Box-Box/QMOF_MPContrib/BatteryMOFs_filter/Structures_all
/Users/matthiasgolomb/Library/CloudStorage/Box-Box/QMOF_MPContrib/BatteryMOFs_filter/Structures_all/QMOF/Ni4H24C24N16/Pristine/Relaxation
/Users/matthiasgolomb/Library/CloudStorage/Box-Box/QMOF_MPContrib/BatteryMOFs_filter/Structures_all
/Users/matthiasgolomb/Library/CloudStorage/Box-Box/QMOF_MPContrib/BatteryMOFs_filter/Structures_all/QMOF/Mn2P2H32C52N4O12/Pristine/Relaxation
/Users/matthiasgolomb/Library/CloudStorage/

In [4]:
runner = OximachineRunner()

Make spin predictor

In [5]:
# First create dictionary for d orbitals

d_dict = {'d0':[0], 'd1':[1,-1], 'd2':[2,0,-2], 'd3':[3,1,-1,-3], 'd4':[4,2,0,-2,-4], 'd5':[5,3,1,-1,-3,-5], 
          'd6':[4,2,0,-2,-4], 'd7':[3,1,-1,-3], 'd8':[2,0,-2], 'd9':[1,-1], 'd10':[0]}

# Now create dictionary for species and their oxidation state

K_dict = {'1':'d0'}

Ca_dict = {'2':'d0'}

Sc_dict = {'3':'d0'}

Ti_dict = {'4':'d0'}

V_dict = {'2':'d3', '3':'d2', '4':'d1', '5':'d0'}

Cr_dict = {'2':'d4', '3':'d3', '6':'d0'}

Mn_dict = {'2':'d5', '3':'d4', '4':'d3', '6':'d1', '7':'d0'}

Fe_dict = {'2':'d6', '3':'d5'}

Co_dict = {'2':'d7', '3':'d6'}

Ni_dict = {'2':'d8'}

Cu_dict = {'1':'d10', '2':'d9'}

Zn_dict = {'2':'d10'}

Pt_dict = {'2':'d8'}

# Assign species to dicts in another dict

species_dict = {'K':K_dict, 'Ca':Ca_dict, 'Sc':Sc_dict, 'Ti':Ti_dict, 'V':V_dict, 'Cr':Cr_dict, 'Mn':Mn_dict, 
                'Fe':Fe_dict, 'Co':Co_dict, 'Ni':Ni_dict, 'Cu':Cu_dict, 'Zn':Zn_dict, 'Pt':Pt_dict}

In [251]:
for folder in os.listdir('./Results_nospin/QMOF/'):
    
    # Read structure and run oximachine
    
    struct = read('./Results_nospin/QMOF/{folder}/Pristine/Relaxation/geometry.in'.format(folder=folder))
    
    print('Running {folder}'.format(folder=folder))

    species_list = runner.run_oximachine(struct)['metal_symbols']
    indice_list = runner.run_oximachine(struct)['metal_indices']
    oxistate_list = runner.run_oximachine(struct)['prediction']
    
    print(species_list, oxistate_list)

    # Find permutations of possible spin moments on magnetic species

    spin_state_lst = []

    for species, oxistate in zip(species_list, oxistate_list):
        loop_spec_dict = species_dict[species]
        delecs = loop_spec_dict[str(oxistate)]
        spin_states = d_dict[str(delecs)]
        spin_state_lst.append(spin_states)

    perms_temp = list(itertools.product(*spin_state_lst))

    perms = []
    for perm in perms_temp:
        if tuple(x*(-1) for x in perm) not in perms:
            perms.append(perm)

    for perm in perms:
        for index, spin_mag in zip(indice_list, perm):
            struct[index].magmom = spin_mag
    
    os.environ['AIMS_SPECIES_DIR'] = '/Users/matthiasgolomb/Codes/FHIaims/species_defaults/defaults_2020/light'

    calc = Aims(xc='pbesol',
    #            hybrid_xc_coeff=0.5,
    #        hse_unit='B',
            sc_accuracy_etot=1e-4,
            sc_accuracy_eev=1e-2,
            sc_accuracy_rho=1e-5,
            sc_accuracy_forces=1e-2,
            relax_geometry=('trm', 1e-2),
            relax_unit_cell='full',
            collect_eigenvectors='.false.',
            spin='collinear',
    #            charge=-1,
    #            fixed_spin_moment=1.0,
                default_initial_moment=0.0,
            relativistic=('atomic_zora','scalar'),
            k_grid=(1,1,1))

    for perm in perms:
        for index, spin_mag in zip(indice_list, perm):
            struct[index].magmom = spin_mag
        if not os.path.exists('Calculations_spin'):
            os.makedirs('Calculations_spin')
        if not os.path.exists('Calculations_spin/{formula}'.format(formula=struct.symbols)):
            os.makedirs('Calculations_spin/{formula}'.format(formula=struct.symbols))
        if not os.path.exists('Calculations_spin/{formula}/{perm_name}'.format(formula=struct.symbols, 
                                                                          perm_name=''.join(map(str, perm)))):
            os.makedirs('Calculations_spin/{formula}/{perm_name}'.format(formula=struct.symbols, 
                                                                          perm_name=''.join(map(str, perm))))

        os.chdir('Calculations_spin/{formula}/{perm_name}'.format(formula=struct.symbols, 
                                                                          perm_name=''.join(map(str, perm))))
        struct.cal = calc
        calc.write_input(struct)
        os.chdir('../../../')

Running K2Cr2H12C28N4O16
['K', 'K', 'Cr', 'Cr'] [1, 1, 3, 3]
Running Cu2H24C40N4O8
['Cu', 'Cu'] [1, 1]
Running CoH8C12N2O4
['Co'] [2]
Running Co4H28C40N8Cl4O8
['Co', 'Co', 'Co', 'Co'] [2, 2, 2, 2]
Running MnH14C14N12O4
['Mn'] [3]
Running Fe2H24C28N4O12
['Fe', 'Fe'] [2, 2]
Running Cu4H16C40N12O12
['Cu', 'Cu', 'Cu', 'Cu'] [1, 1, 1, 1]
Running Cu4H16C40N24O8
['Cu', 'Cu', 'Cu', 'Cu'] [2, 2, 2, 2]
Running Fe2H6C8N16
['Fe', 'Fe'] [2, 2]
Running Sc4H28C36N8O24
['Sc', 'Sc', 'Sc', 'Sc'] [3, 3, 3, 3]
Running Ti2H52C68N4O8
['Ti', 'Ti'] [4, 4]
Running VH8C10N2F3
['V'] [3]
Running V6Cu2H18C20N6O17
['V', 'V', 'V', 'V', 'V', 'V', 'Cu', 'Cu'] [5, 5, 5, 5, 5, 5, 2, 2]
Running Cu4H40C64N16O16
['Cu', 'Cu', 'Cu', 'Cu'] [2, 2, 2, 2]
Running Co4H12C8N20O12
['Co', 'Co', 'Co', 'Co'] [2, 2, 2, 2]
Running Co3H26C40N4O10
['Co', 'Co', 'Co'] [2, 2, 2]
Running Co2H22C36N6O12
['Co', 'Co'] [2, 2]
Running Ca2V2P4H28C24O18
['Ca', 'Ca', 'V', 'V'] [2, 2, 4, 4]
Running Co4H12C32Cl4O16
['Co', 'Co', 'Co', 'Co'] [2, 2, 2, 2]

In [250]:
os.chdir('/Users/matthiasgolomb/Library/CloudStorage/Box-Box/Results/MOFs/BatteryMOFs/MOF_Highthroughput/QMOF/Structures_all')

In [78]:
for folder in os.listdir('./Results_spin/Calculations_spin/'):
    for subfolder in os.listdir('./Results_spin/Calculations_spin/{folder}'.format(folder=folder)):
        print(folder, subfolder)
            
        min_structure = ''
        min_energy = 0

        with open('./Results_spin/Calculations_spin/{folder}/{subfolder}/aims_craynov21.out'.format(folder=folder, subfolder=subfolder), 'r') as f:
            lines = f.readlines()
            print(lines[-2])
            if 'Have a nice day' in lines[-2]:
                print('Reading', folder, subfolder, lines[-2])
                struct_result = read('./Results_spin/Calculations_spin/{folder}/{subfolder}/aims_craynov21.out'.format(folder=folder, subfolder=subfolder))
                print(folder, struct_result.get_total_energy(), '{subfolder}'.format(subfolder=subfolder))
                if struct_result.get_total_energy() < min_energy:
                    min_energy = struct_result.get_total_energy()
                    min_structure = '{subfolder}'.format(subfolder=subfolder)
            else:
                pass

    print('The minimum is', folder, min_energy, min_structure)

K2Cr2H12C28N4O16 0033
          Have a nice day.

Reading K2Cr2H12C28N4O16 0033           Have a nice day.

K2Cr2H12C28N4O16 0011
          Have a nice day.

Reading K2Cr2H12C28N4O16 0011           Have a nice day.

K2Cr2H12C28N4O16 001-1
          Have a nice day.

Reading K2Cr2H12C28N4O16 001-1           Have a nice day.

K2Cr2H12C28N4O16 003-3
          Have a nice day.

Reading K2Cr2H12C28N4O16 003-3           Have a nice day.

K2Cr2H12C28N4O16 0031
          Have a nice day.

Reading K2Cr2H12C28N4O16 0031           Have a nice day.

K2Cr2H12C28N4O16 0013
          Have a nice day.

Reading K2Cr2H12C28N4O16 0013           Have a nice day.

K2Cr2H12C28N4O16 003-1
          Have a nice day.

Reading K2Cr2H12C28N4O16 003-1           Have a nice day.

K2Cr2H12C28N4O16 001-3
          Have a nice day.

Reading K2Cr2H12C28N4O16 001-3           Have a nice day.

Cu2H24C40N4O8 00
          Have a nice day.

Reading Cu2H24C40N4O8 00           Have a nice day.

CoH8C12N2O4 1


CoH8C12N2O4 3


KeyboardInterrupt: 

In [9]:
for folder in os.listdir('Calculations_spin_minima/'):
    if os.path.isdir('Calculations_spin_minima/{folder}'.format(folder=folder)):
        
        struct_in = read('Calculations_spin_minima/{folder}/geometry.in'.format(folder=folder))
        struct_out = read('temp/{folder}/geometry.in'.format(folder=folder))
        struct_out.set_initial_magnetic_moments(struct_in.get_initial_magnetic_moments())
#        print(struct_in[0], struct_out[0])
        
        if not os.path.exists('temp2/{folder}'.format(folder=folder)):
            os.makedirs('temp2/{folder}'.format(folder=folder))
        
        os.environ['AIMS_SPECIES_DIR'] = '/Users/matthias/Results_loc/Codes/FHIaims/species_defaults/defaults_2020/intermediate'
        
        cubefile = AimsCube(plots=('hartree_potential', 'spin_density'))

        calc = Aims(
                xc=('hse06', 0.11),
                vdw_correction_hirshfeld='',
        #            hybrid_xc_coeff=0.5,
                hse_unit='B',
                sc_accuracy_etot=1e-4,
                sc_accuracy_eev=1e-2,
                sc_accuracy_rho=1e-5,
                sc_accuracy_forces=1e-2,
        #        relax_geometry=('trm', 1e-2),
        #        relax_unit_cell='full',
                collect_eigenvectors='.false.',
                spin='collinear',
        #            charge=-1,
        #            fixed_spin_moment=1.0,
                    default_initial_moment=0.0,
                relativistic=('atomic_zora','scalar'),
                k_grid=(1,1,1),
                cubes=cubefile)
        
        os.chdir('temp2/{folder}'.format(folder=folder))
        struct_out.calc = calc
        calc.write_input(struct_out)
        os.chdir('../../')